In [61]:
import pandas as pd
import pandas as np
import xgboost as xgb
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split

In [4]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 500)

height has been deprecated.



In [12]:
train = pd.read_csv(r"C:\Users\Mayank\Desktop\Kaggle\Toxic Comment Classification\\train.csv")
test = pd.read_csv (r"C:\Users\Mayank\Desktop\Kaggle\Toxic Comment Classification\test.csv")
submission = pd.read_csv(r"C:\Users\Mayank\Desktop\Kaggle\Toxic Comment Classification\sample_submission.csv")

In [17]:
#Checking the initial train shape
train.shape

(159571, 8)

In [16]:
#Test initial shape
test.shape

(153164, 2)

In [18]:
#train columns
train.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [43]:
#Let's get the ratio of 1's to 0's
train["identity_hate"].value_counts()

0    158166
1    1405  
Name: identity_hate, dtype: int64

In [44]:
#ratio
a = (1405/159571)*100
print (a)

0.8804858025581089


In [30]:
#So the class is imalanced , the highest being toxic

In [45]:
#Features.Time to create some basic features before feeding into model.

In [48]:
#let's fill the na's in the data
train = train.fillna(' ')
test = test.fillna(' ')

In [54]:
#Combining text to create features
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

In [51]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)

In [55]:
word_vectorizer.fit(all_text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents='unicode', sublinear_tf=True,
        token_pattern='\\w{1,}', tokenizer=None, use_idf=True,
        vocabulary=None)

In [56]:
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

C:\Users\Mayank\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [59]:
# #best parameters for XGB
# param['objective'] = 'binary:logistic'

# param['eta'] = 0.095

# param['max_depth'] = 5

# param['silent'] = 1

# param['eval_metric'] = 'auc'

# param['min_child_weight'] = 2

# param['subsample'] = 0.7

# param['colsample_bytree'] = 0.7

In [62]:
#Splitting the data into train and test sets using SVD feature
y_train = train['toxic']
x_train, x_valid, y_train, y_valid = train_test_split(train_word_features, y_train, test_size=0.10)

In [63]:
n_folds = 10
early_stopping = 10
params = {'eta': 0.095, 'max_depth': 5, 'subsample': 0.7, 'colsample_bytree': 0.7,'min_child_weight':2, 'objective': 'binary:logistic', 'seed': 99, 'silent': 1, 'eval_metric':'auc', 'nthread':4}

xg_train = xgb.DMatrix(x_train,  label=y_t rain);

cv = xgb.cv(params, xg_train, 500, nfold=n_folds, early_stopping_rounds=early_stopping, verbose_eval=1)


[0]	train-auc:0.652526+0.0220688	test-auc:0.650769+0.021384
[1]	train-auc:0.688221+0.0129354	test-auc:0.686672+0.0161697
[2]	train-auc:0.702571+0.0118533	test-auc:0.700593+0.0141655
[3]	train-auc:0.715843+0.00825256	test-auc:0.714989+0.0118694
[4]	train-auc:0.721323+0.00729968	test-auc:0.719717+0.0102163
[5]	train-auc:0.725834+0.0065551	test-auc:0.724713+0.00836576
[6]	train-auc:0.732606+0.0060704	test-auc:0.731209+0.00800191
[7]	train-auc:0.736327+0.00438232	test-auc:0.734656+0.00831625
[8]	train-auc:0.74093+0.00423365	test-auc:0.739646+0.00870024
[9]	train-auc:0.74408+0.00424988	test-auc:0.743097+0.00989875
[10]	train-auc:0.745245+0.00377992	test-auc:0.743999+0.0103901
[11]	train-auc:0.749074+0.00213481	test-auc:0.748314+0.00617721
[12]	train-auc:0.753403+0.0128617	test-auc:0.752541+0.0116692
[13]	train-auc:0.756715+0.0134993	test-auc:0.754959+0.012556
[14]	train-auc:0.763443+0.0122786	test-auc:0.760955+0.0115522
[15]	train-auc:0.777864+0.0182137	test-auc:0.775087+0.0208275
[16]	trai

In [64]:
def importance_XGB(model):
    impdf = []
    for ft, score in model.get_fscore().iteritems():
        impdf.append({'feature': ft, 'importance': score})
    impdf = pd.DataFrame(impdf)
    impdf = impdf.sort_values(by='importance', ascending=False).reset_index(drop=True)
    impdf['importance'] /= impdf['importance'].sum()
    return impdf

In [67]:
cv

,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
0,0.650769,0.021384,0.652526,0.022069
1,0.686672,0.016170,0.688221,0.012935
2,0.700593,0.014165,0.702571,0.011853
3,0.714989,0.011869,0.715843,0.008253
4,0.719717,0.010216,0.721323,0.007300
5,0.724713,0.008366,0.725834,0.006555
6,0.731209,0.008002,0.732606,0.006070
7,0.734656,0.008316,0.736327,0.004382
8,0.739646,0.008700,0.740930,0.004234
9,0.743097,0.009899,0.744080,0.004250
